# РАСЧЕТ ПЛАНА ПРОИЗВОДСТВА НА 2025 г.

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Загрузка исходных данных для расчета

In [74]:
actual_items = pd.read_excel('data/actual_items.xlsx')
current_stocks = pd.read_excel('data/stocks_2024.xlsx') # Остатки на 31.12.2024
current_stocks['item'] = current_stocks['item'].str.replace(', шт', '')
current_stocks.fillna(0, inplace=True)
current_stocks['stock_level'] = current_stocks['stock_level'].astype('int')
sales_plan = pd.read_excel('data/sales_plan_2025.xlsx')

In [75]:
sales_plan['date'] = pd.to_datetime(sales_plan['date'])

## Расчет плана производства

Функции для расчета:

In [76]:
def calculate_production(actual_items, stocks, sales_plan, safety_level=41, working_days=20, start_date="2024-10-01"):
    production_plan = {}
    actual_items = actual_items['item'].to_list()
    sales_plan = sales_plan.query('date == @start_date')
    for x in actual_items:
        if x in stocks['item'].to_list():
            stock_level = stocks.query('item == @x')['stock_level'].tolist()[0]
        else: 
            stock_level = 0
        if x in sales_plan['item'].to_list():
            sales_quantity = sales_plan.query('item == @x')['quantity'].tolist()[0]
        else:
            sales_quantity = 1
        sales_per_day = sales_quantity / working_days
        if (stock_level / sales_per_day - safety_level) >0:
            production_plan[x] = 0
        else:
            production_plan[x] = round((safety_level - stock_level / sales_per_day) * sales_per_day, -3)
    return production_plan

In [77]:
def calculate_stocks(actual_items, stocks, sales_plan, production_plan, start_date="2024-10-01"):
    sales_plan = sales_plan.query('date == @start_date')
    df = (actual_items
          .merge(stocks, how='left', on='item')
          .merge(sales_plan, how='left', on='item')
          .merge(production_plan, how='left', on='item'))
    df['stock_level'] = df['stock_level'] + df['production_volume'] - df['quantity']
    return df[['item', 'stock_level']]

Расчет плана производства:

In [98]:
planning_period = ['2025-01-01', '2025-02-01', '2025-03-01', '2025-04-01', '2025-05-01', 
                   '2025-06-01', '2025-07-01', '2025-08-01', '2025-09-01', '2025-10-01', '2025-11-01', '2025-12-01']
safety_level = [56, 54, 51, 69, 78, 66, 79, 69, 50, 57, 55, 52]
production_plan = {}
stocks = current_stocks
planning_stocks = []
for period, level in zip(planning_period, safety_level):
    plan = calculate_production(actual_items, stocks, sales_plan, safety_level=level, start_date=period)
    production_plan[period] = plan
    plan = pd.DataFrame.from_dict([plan]).T.reset_index().rename(columns={'index':'item', 0:'production_volume'})
    new_stocks = calculate_stocks(actual_items, stocks, sales_plan, plan, start_date=period)
    planning_stocks.append(new_stocks)
    stocks = new_stocks

## ИТОГОВЫЙ ПЛАН ПРОИЗВОДСТВА

In [99]:
final_production_plan = pd.DataFrame(production_plan).reset_index().rename(columns={'index':'item'})
final_production_plan.head()

,item,2025-01-01,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01,2025-08-01,2025-09-01,2025-10-01,2025-11-01,2025-12-01
0,Аир корневища 75г,5000.0,7000.0,8000.0,4000.0,4000.0,6000.0,4000.0,5000.0,9000.0,7000.0,8000.0,9000.0
1,Алтей корни 75г,0.0,0.0,0.0,0.0,1000.0,2000.0,2000.0,1000.0,3000.0,3000.0,3000.0,3000.0
2,Багульник болотный побеги 50г,5000.0,9000.0,8000.0,5000.0,5000.0,7000.0,4000.0,6000.0,10000.0,8000.0,10000.0,10000.0
3,Береза почки 50г,6000.0,9000.0,9000.0,5000.0,5000.0,7000.0,4000.0,6000.0,11000.0,8000.0,10000.0,11000.0
4,Бессмертник песчаный цветки 30г,19000.0,17000.0,17000.0,10000.0,10000.0,13000.0,8000.0,13000.0,20000.0,17000.0,19000.0,21000.0


In [100]:
sales_by_month = sales_plan.groupby(by='date').sum().quantity.reset_index()
production_plan_by_month = final_production_plan.sum().drop('item',axis=0).reset_index().rename(columns={'index':'date', 0:'production_volume'})
production_plan_by_month['date'] = pd.to_datetime(production_plan_by_month['date'])
production_plan_by_month.merge(sales_by_month, how='left', on='date')

,date,production_volume,quantity
0,2025-01-01,2674000.0,2629357
1,2025-02-01,2930000.0,2945739
2,2025-03-01,3116000.0,3266885
3,2025-04-01,1882000.0,2066966
4,2025-05-01,1798000.0,1800563
5,2025-06-01,2322000.0,2305332
6,2025-07-01,1498000.0,1712985
7,2025-08-01,2187000.0,2203110
8,2025-09-01,3482000.0,3605595
9,2025-10-01,3005000.0,3031800


In [101]:
production_plan_by_month.production_volume.sum()

31976000.0

## ИТОГОВЫЙ ЭКСПОРТ В EXCEL

In [102]:
final_production_plan.to_excel('data/final_production_plan_2.xlsx')